# Results
In this notebook, we define and calculate the statistics of our tests.

In [46]:
# load data

import json
import pandas as pd
from collections import defaultdict
import math
import os

def loadData(fileName):
    '''Loading data from result files'''
    with open(f'{fileName}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def normalize(cases):
    '''make all the ddx list of a case of the same length by padding with None'''
    for case in cases.values():
        maxLen = max(len(result) for result in case.values())
        for result in case.values():
            result += [None]*(maxLen-len(result))

        assert len(set(len(result) for result in case.values())) == 1
    
    return cases

def getDataframe(case):
    '''Convert each test case into a dataframe'''
    return pd.DataFrame(
        case,
        columns=['gs',*sorted([key for key in case.keys() if key != 'gs'])],
        index= list(range(1,1+len(list(case.values())[0])))
        )


In [47]:
# We need to make all the differentials of the same length to ease comparison
# We pad the lists with None
data = loadData('allResults')
# data = loadData('aveyAll')
for caseNum, tests in data.items():
    assert len(tests.keys()) == 9, f"app missing in case {caseNum}, {tests.keys()}"
normalizedData = normalize(data)
cases = {int(id): getDataframe(case) for id, case in normalizedData.items()}
caseClassification = loadData('case-classification')
f'We have {len(cases)} cases in the experiment.'


'We have 400 cases in the experiment.'

Let us have a look at all the cases.

In [48]:
from IPython.display import display
for caseNum, case in cases.items():
    print(f"Case number {caseNum}")
    display(case)

Case number 1


,gs,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,WebMD
1,acute bronchitis,acute bronchitis,None,acute bronchitis,acute bronchitis,gastroesophageal reflux,reactive airway disease with superimposed infe...,post infectious cough,asthma
2,common cold,asthma,None,asthma,asthma,post nasal drip,None,acute bronchitis,acute bronchitis
3,influenza,whooping cough,None,benign cough,pneumonia,acute bronchitis,None,None,coronavirus disease 2019
4,asthma,pneumonia,None,None,common cold,None,None,None,influenza
5,None,mycoplasma pneumonia,None,None,None,None,None,None,chronic sinusitis


Case number 2


,gs,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,WebMD
1,unstable angina,unstable angina,None,None,myocardial infarction,stable angina,unstable angina,myocardial infarction,myocardial infarction
2,myocardial infarction,collapsed lung,None,None,musculoskeletal chest pain,unstable angina,myocardial infarction,unstable angina,broken ribs
3,stable angina,myocardial infarction,None,None,cardia arrhythmias,myocardial infarction,None,heart failure,costochondritis
4,pericardial effusion,chronic obstructive pulmonary disease,None,None,unstable angina,None,None,None,heartburn/gerd
5,None,stable angina,None,None,None,None,None,None,unstable angina
6,None,None,None,None,None,None,None,None,pulmonary embolism


Case number 3


...

,gs,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,WebMD
1,pre-eclampsia,tension headache,None,None,dehydration,reduse volume of amenutic fluids,pre-eclampsia,pre-eclampsia,ectopic pregnancy
2,gestational hypertension,migraine,None,None,migraine,fetal distress,gestational hypertension,gestational hypertension,heartburn
3,None,non specific headache,None,None,general headache,pre-eclampsia,None,peptic ulcer disease,morning sickness
4,None,nonpyogenic thrombosis of the intracranial ven...,None,None,None,None,None,hepatitis,miscarriage
5,None,leptosperosis,None,None,None,None,None,cva,pregnancy
6,None,None,None,None,None,None,None,None,cholelithiasis
7,None,None,None,None,None,None,None,None,acute gastroenteritis


Case number 499


,gs,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,WebMD
1,lymphedema,lymphedema,None,atrial fibrillation,deep venous thrombosis,insect bite,deep venous thrombosis,elephantiasis / filariasis,deep venous thrombosis
2,deep venous thrombosis,venous insufficiency,None,None,lymphedema,deep venous thrombosis,baker's cyst rupture,lymphedema,thrombophlebitis
3,None,None,None,None,None,lymphedema,None,None,cellulitis
4,None,None,None,None,None,fluids stasis,None,None,pulmonary embolism
5,None,None,None,None,None,None,None,None,baker's cyst


Case number 500


,gs,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,WebMD
1,tonsillopharyngitis,tonsillopharyngitis,None,possible coronavirus,None,common cold,viral upper respiratory tract infection,mononucleosis,tonsillopharyngitis
2,mononucleosis,common cold,None,None,None,tonsillopharyngitis,None,tonsillopharyngitis,peritonsillar abscess
3,influenza,influenza,None,None,None,None,None,coronavirus disease 2019,common cold
4,None,None,None,None,None,None,None,None,mononucleosis


## Let us define the metrics now.

### Terms used
- TP: True positive (correct disease retrieved)
- TN: True negative (wrong disease **not** retrieved)
- FP: False positive (wrong disease retrieved)
- FN: False negative (correct disease **not** retrieved)
- gold standard - the correct list of diseases as determined by collective intelligence of doctors

### Precision
Precision helps us understand how exact our results are. It gives us an intuition about how many wrong diseases (false positives) are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the complete list retrieved*.
$$precision = \frac{TP}{TP + FP} = \frac{TP}{\text{length of differential list}}$$

### Recall
Recall is a measure of how many of the correct diseases are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the gold standard list*.
$$recall = \frac{TP}{TP + FN} = \frac{TP}{\text{length of the gold standard}}$$

### F1 Score
F1 score is the weighted harmonic mean of *precision* and *recall*. It is a metric that combines *precision* and *recall* and gives us 1 score for easier comparison.

Suppose $\beta$ defines how important is $recall$ to $precision$ then,
$$fscore_{\beta} = (1 + \beta^2)\frac{precision \cdot recall}{(\beta^2 \cdot precision) + recall}$$
Substituting $\beta = 1$,
$$fscore_{1} = \frac{2 \cdot precision \cdot recall}{ precision + recall}$$

### NDCG
NDCG or Normalized Discounted Cumulative Gain is measure of how accurate the ranking is. In our calculations, we use
$$DCG = \sum_{i=1}^n\frac{2^{relevance_i}-1}{log_2(i+1)}$$
where $n$ is the number of differentials in the returned list and  
$relevance_i = |gold standard| - rank_{gold\ standard}(ddx[i])$ if $ddx[i]$ is present, 0 otherwise.

$$NDCG = \frac{DCG_{ddx}}{DCG_{gold\ standard}}$$

### M Score
M Score determines where the top disease (gold standard) appears in the returned differential.
$$M_i = \text{gold standard[0]} \in \text{ddx[:i]}$$

### Position
Shows the position of the gold standard[0] in the returned differential. 

### Length
$$length = \frac{|ddx|}{|gold\ standard|}$$


In [49]:
import math

beta = 1 #used in recall

def getPrecision(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp == 0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()


def getF1Score(precision: float, recall: float, beta: float = 1) -> float:
    return math.nan if precision+recall == 0 else \
        (1+beta**2)*precision*recall/(precision*(beta**2)+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score: float, index: int) -> float:
        '''The index is 1 based'''
        return (math.pow(2, score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i], i+1) for i in range(len(scores)))

    candidateRelevance = []
    goldStandard = list(goldStandard)
    for index, disease in enumerate(candidate):
        if disease is not None and disease in goldStandard:
            candidateRelevance.append(
                discount(scores[goldStandard.index(disease)], index+1))
        else:
            candidateRelevance.append(0)

    return sum(candidateRelevance)/maxDCG


def getMScore(goldStandard: pd.Series, candidate: pd.Series, m=1) -> bool:
    return goldStandard.values[0] in candidate.values[:m]


def getPosition(goldStandard: pd.Series, candidate: pd.Series) -> bool:
    return math.nan if goldStandard.values[0] not in candidate.values else\
        1 + list(candidate.values).index(goldStandard.values[0])


def getLength(goldStandard: pd.Series, candidate: pd.Series) -> int:
    return math.nan if candidate.count() == 0 else \
        candidate.count()/goldStandard.count()


def getScoresCase(case: pd.DataFrame, beta:float = 1) -> pd.DataFrame:
    scores = [
        [getPrecision(case.iloc[:, 0], case.iloc[:, i])
         for i in range(1, len(case.columns))],
        [getRecall(case.iloc[:, 0], case.iloc[:, i])
         for i in range(1, len(case.columns))]
    ]

    scores.append([getF1Score(scores[0][i], scores[1][i],beta=beta)
                  for i in range(len(case.columns)-1)])
    
    # relevance for a list of 4 is 4, 3, 2, 1
    # relevance for a list of 2 is 2, 1
    scores.append([getNDCG(case.iloc[:, 0], case.iloc[:, i],
                           list(range(case.iloc[:, 0].count(), 0, -1)))
                   for i in range(1, len(case.columns))])

    for m in range(1, 6, 2):
        scores.append([getMScore(case.iloc[:, 0], case.iloc[:, i], m)
                      for i in range(1, len(case.columns))])

    scores.append([getPosition(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    scores.append([getLength(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    return pd.DataFrame(scores, columns=case.columns[1:],
                        index=["precision", "recall", "f1-score", "NDCG",
                               "M1", "M3", "M5", "position", "length (x of gs)"],
                        )

scores = {id:getScoresCase(case) for id, case in cases.items()}

Let us define the experiment now. We will pick which cases to compute startistics for.

In [50]:
from collections import defaultdict
experiments = {}
combineLabels = defaultdict(list)

def addExperiment(caseType,casesToConsider):
    #add experiment to ignore no case
    experiments[f'ignore_{caseType}_none'] = set(casesToConsider)

    for app, classifications in caseClassification['apps'].items():
        for classification, caseNums in classifications.items():
            #record labels to combine later
            combineLabels[f'ignore_{caseType}_{classification}'].append(f'ignore_{caseType}_{app}_{classification}')
            #experiment to ignore the cases for the particular app
            experiments[f'ignore_{caseType}_{app}_{classification}'] = set(casesToConsider) - set(caseNums)
            #experiment to ignore the cases for all the apps
            if f'ignore_{caseType}_any_{classification}' in experiments:
                experiments[f'ignore_{caseType}_any_{classification}'] =\
                 experiments[f'ignore_{caseType}_any_{classification}'] - set(caseNums)
            else:
                experiments[f'ignore_{caseType}_any_{classification}'] = experiments[f'ignore_{caseType}_{app}_{classification}']

addExperiment('common',caseClassification['common'])
addExperiment('less-common',caseClassification['less common'])
addExperiment('all-cases',set(caseClassification['less common']) | set(caseClassification['common']))
    

list(enumerate(experiments.keys()))


[(0, 'ignore_common_none'),
 (1, 'ignore_common_Ada_session_failed'),
 (2, 'ignore_common_any_session_failed'),
 (3, 'ignore_common_Ada_no_disease_found'),
 (4, 'ignore_common_any_no_disease_found'),
 (5, 'ignore_common_Babylon_session_failed'),
 (6, 'ignore_common_Babylon_no_disease_found'),
 (7, 'ignore_common_Buoy_session_failed'),
 (8, 'ignore_common_Buoy_no_disease_found'),
 (9, 'ignore_common_K health_session_failed'),
 (10, 'ignore_common_K health_no_disease_found'),
 (11, 'ignore_common_WebMD_session_failed'),
 (12, 'ignore_common_WebMD_no_disease_found'),
 (13, 'ignore_less-common_none'),
 (14, 'ignore_less-common_Ada_session_failed'),
 (15, 'ignore_less-common_any_session_failed'),
 (16, 'ignore_less-common_Ada_no_disease_found'),
 (17, 'ignore_less-common_any_no_disease_found'),
 (18, 'ignore_less-common_Babylon_session_failed'),
 (19, 'ignore_less-common_Babylon_no_disease_found'),
 (20, 'ignore_less-common_Buoy_session_failed'),
 (21, 'ignore_less-common_Buoy_no_disease_fo

In [51]:
def getAverage(scores, row: int, col: int) -> float:
    values = []
    for score in scores.values():
        if not math.isnan(score.iloc[row, col]):
            values.append(score.iloc[row, col])

    return 0 if not values else round(sum(values)/len(values), 3)

results = {}
for label, casesToConsider in experiments.items():
    selectedScores = {id:score for id,score in scores.items() if int(id) in casesToConsider}
    columns = next(iter(scores.values())).columns
    averageScores = pd.DataFrame(
        [
            [
                getAverage(selectedScores, row, col)
                for col in range(len(columns))
            ]
            for row in range(9)
        ],
        columns=columns,
        index=[
            f"average_{x}" for x in [
                "precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length (x of gs)"
            ]
        ]
    )

    # ignore nan for recall
    for col in next(iter(scores.values())).columns:
        p = averageScores.loc["average_precision", col]
        r = averageScores.loc["average_recall", col]
        averageScores.loc["average_f1-score",
                        col] = round(getF1Score(p, r, beta), 3)

    doctorResults = averageScores.loc[:, "Mohmmad Almadani"] +\
        averageScores.loc[:, "Noor Joudeh"] +\
        averageScores.loc[:,
                            "Tala Hammouri"]
    doctorResults /= 3.0
    
    averageScores.insert(
        loc=7, column="average_doctor",
        value=doctorResults.round(3),
    )

    results[label] = averageScores


Let us print all the results. The experiments are nomenclatured as follows:
- **[common|uncommon|all]:** means whether only common cases were considered, only uncommon cases were considered, and so on.
- **app name / any:** If an app name is present, then we ignore only those cases that pertain to it. If the label is *any* then we consider all apps in that experiment.
- **failure type**: The apps can fail in 2 ways. Either a session does not complete due to some reason or the app fails to retrieve any diagnosis. If this is set to None, then we ignore the failures and consider all cases under option 1 above.

In [52]:
def displayResults(results,printNumCases=True):
    for label, result in results.items():
        result.to_csv(f'stats/{label}.json',sep=';')
        if printNumCases:
            print(f'Results for experiment {label}, which has {len(set(experiments[label]) & set(scores.keys()))} cases, is')
        else:
            print(f'Results for experiment {label} is')
        display(result)

displayResults({key:val for key, val in results.items() if 'any' not in key})

Results for experiment ignore_common_none, which has 222 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.488,0.054,0.360,0.384,0.628,0.797,0.743,0.723,0.262
average_recall,0.633,0.032,0.346,0.405,0.433,0.404,0.557,0.465,0.532
average_f1-score,0.551,0.040,0.353,0.394,0.513,0.536,0.637,0.562,0.351
average_NDCG,0.718,0.038,0.409,0.468,0.582,0.604,0.736,0.641,0.503
average_M1,0.640,0.036,0.351,0.410,0.568,0.662,0.752,0.661,0.329
average_M3,0.833,0.041,0.514,0.527,0.712,0.721,0.883,0.772,0.541
average_M5,0.874,0.045,0.514,0.550,0.716,0.721,0.887,0.775,0.649
average_position,1.438,1.400,1.386,1.410,1.252,1.100,1.178,1.177,2.147
average_length (x of gs),1.486,0.567,1.008,1.194,0.853,0.558,0.856,0.756,2.217


Results for experiment ignore_common_Ada_session_failed, which has 222 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.488,0.054,0.360,0.384,0.628,0.797,0.743,0.723,0.262
average_recall,0.633,0.032,0.346,0.405,0.433,0.404,0.557,0.465,0.532
average_f1-score,0.551,0.040,0.353,0.394,0.513,0.536,0.637,0.562,0.351
average_NDCG,0.718,0.038,0.409,0.468,0.582,0.604,0.736,0.641,0.503
average_M1,0.640,0.036,0.351,0.410,0.568,0.662,0.752,0.661,0.329
average_M3,0.833,0.041,0.514,0.527,0.712,0.721,0.883,0.772,0.541
average_M5,0.874,0.045,0.514,0.550,0.716,0.721,0.887,0.775,0.649
average_position,1.438,1.400,1.386,1.410,1.252,1.100,1.178,1.177,2.147
average_length (x of gs),1.486,0.567,1.008,1.194,0.853,0.558,0.856,0.756,2.217


Results for experiment ignore_common_Ada_no_disease_found, which has 222 cases, is


...

,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.441,0.030,0.294,0.311,0.583,0.782,0.718,0.694,0.217
average_recall,0.578,0.018,0.286,0.348,0.406,0.408,0.568,0.461,0.445
average_f1-score,0.500,0.022,0.290,0.328,0.479,0.536,0.634,0.550,0.292
average_NDCG,0.627,0.020,0.325,0.360,0.515,0.579,0.731,0.608,0.400
average_M1,0.533,0.018,0.261,0.279,0.490,0.613,0.721,0.608,0.246
average_M3,0.709,0.020,0.402,0.389,0.613,0.673,0.879,0.722,0.412
average_M5,0.759,0.023,0.402,0.415,0.621,0.676,0.882,0.726,0.505
average_position,1.526,1.444,1.450,1.627,1.275,1.119,1.236,1.210,2.215
average_length (x of gs),1.558,0.513,1.036,1.271,0.865,0.571,0.904,0.780,2.279


Results for experiment ignore_all-cases_WebMD_session_failed, which has 397 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.443,0.033,0.295,0.311,0.583,0.779,0.714,0.692,0.218
average_recall,0.581,0.019,0.286,0.347,0.406,0.406,0.568,0.460,0.446
average_f1-score,0.503,0.024,0.290,0.328,0.479,0.534,0.633,0.549,0.293
average_NDCG,0.629,0.022,0.325,0.359,0.514,0.577,0.732,0.608,0.401
average_M1,0.534,0.020,0.262,0.277,0.489,0.610,0.723,0.607,0.247
average_M3,0.710,0.023,0.403,0.388,0.612,0.670,0.882,0.721,0.413
average_M5,0.761,0.025,0.403,0.413,0.620,0.673,0.884,0.726,0.506
average_position,1.526,1.400,1.450,1.630,1.276,1.120,1.236,1.211,2.215
average_length (x of gs),1.559,0.512,1.034,1.272,0.866,0.571,0.908,0.782,2.279


Results for experiment ignore_all-cases_WebMD_no_disease_found, which has 397 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.442,0.030,0.294,0.311,0.582,0.782,0.717,0.694,0.218
average_recall,0.579,0.018,0.285,0.346,0.406,0.408,0.568,0.461,0.446
average_f1-score,0.501,0.022,0.289,0.328,0.478,0.536,0.634,0.549,0.293
average_NDCG,0.626,0.020,0.323,0.359,0.514,0.579,0.731,0.608,0.401
average_M1,0.531,0.018,0.259,0.277,0.489,0.612,0.720,0.607,0.247
average_M3,0.708,0.020,0.401,0.388,0.612,0.673,0.879,0.721,0.413
average_M5,0.758,0.023,0.401,0.413,0.620,0.675,0.882,0.726,0.506
average_position,1.528,1.444,1.453,1.630,1.276,1.119,1.236,1.210,2.215
average_length (x of gs),1.555,0.513,1.034,1.271,0.866,0.571,0.905,0.781,2.279


In [53]:
displayResults({key:val for key, val in results.items() if 'any' in key})

Results for experiment ignore_common_any_session_failed, which has 187 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.495,0.055,0.378,0.435,0.630,0.805,0.727,0.721,0.269
average_recall,0.642,0.032,0.360,0.460,0.432,0.409,0.571,0.471,0.547
average_f1-score,0.559,0.040,0.369,0.447,0.513,0.542,0.640,0.565,0.361
average_NDCG,0.714,0.036,0.438,0.533,0.578,0.600,0.739,0.639,0.510
average_M1,0.610,0.037,0.380,0.465,0.556,0.642,0.749,0.649,0.332
average_M3,0.824,0.037,0.556,0.604,0.711,0.711,0.882,0.768,0.540
average_M5,0.866,0.043,0.556,0.631,0.717,0.711,0.888,0.772,0.647
average_position,1.488,1.375,1.394,1.424,1.276,1.120,1.187,1.194,2.151
average_length (x of gs),1.498,0.588,1.017,1.196,0.850,0.564,0.892,0.769,2.218


Results for experiment ignore_common_any_no_disease_found, which has 27 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.522,0.369,0.327,0.322,0.610,0.784,0.765,0.720,0.246
average_recall,0.679,0.240,0.286,0.372,0.440,0.441,0.486,0.456,0.506
average_f1-score,0.590,0.291,0.305,0.345,0.511,0.564,0.594,0.556,0.331
average_NDCG,0.794,0.279,0.389,0.384,0.587,0.652,0.726,0.655,0.468
average_M1,0.815,0.259,0.370,0.333,0.593,0.704,0.815,0.704,0.259
average_M3,0.889,0.296,0.481,0.407,0.667,0.815,0.852,0.778,0.444
average_M5,0.926,0.333,0.481,0.407,0.667,0.815,0.889,0.790,0.630
average_position,1.240,1.444,1.231,1.182,1.111,1.182,1.167,1.153,2.611
average_length (x of gs),1.475,0.607,0.888,1.362,0.909,0.611,0.751,0.757,2.083


Results for experiment ignore_less-common_any_session_failed, which has 152 cases, is


...

,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.373,0.062,0.177,0.095,0.557,0.656,0.677,0.630,0.115
average_recall,0.443,0.031,0.125,0.135,0.333,0.344,0.526,0.401,0.219
average_f1-score,0.405,0.041,0.147,0.112,0.417,0.451,0.592,0.487,0.151
average_NDCG,0.431,0.017,0.108,0.066,0.305,0.382,0.713,0.467,0.123
average_M1,0.250,0.000,0.062,0.000,0.188,0.312,0.688,0.396,0.000
average_M3,0.375,0.000,0.125,0.062,0.250,0.312,0.875,0.479,0.062
average_M5,0.438,0.000,0.125,0.062,0.250,0.312,0.875,0.479,0.125
average_position,2.000,0.000,2.000,3.000,1.250,1.000,1.214,1.155,3.500
average_length (x of gs),1.620,0.380,0.863,1.639,0.802,0.583,0.833,0.739,2.214


Results for experiment ignore_all-cases_any_session_failed, which has 339 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.455,0.033,0.303,0.353,0.590,0.778,0.706,0.691,0.225
average_recall,0.596,0.019,0.295,0.396,0.409,0.409,0.579,0.466,0.461
average_f1-score,0.516,0.024,0.299,0.373,0.483,0.536,0.636,0.552,0.302
average_NDCG,0.634,0.021,0.344,0.410,0.519,0.569,0.731,0.606,0.408
average_M1,0.522,0.021,0.280,0.316,0.496,0.590,0.711,0.599,0.248
average_M3,0.714,0.021,0.431,0.445,0.617,0.661,0.873,0.717,0.419
average_M5,0.764,0.024,0.431,0.475,0.625,0.664,0.876,0.722,0.510
average_position,1.548,1.375,1.445,1.642,1.274,1.142,1.242,1.219,2.228
average_length (x of gs),1.553,0.526,1.054,1.272,0.858,0.580,0.929,0.789,2.271


Results for experiment ignore_all-cases_any_no_disease_found, which has 43 cases, is


,Ada,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,average_doctor,WebMD
average_precision,0.467,0.255,0.271,0.237,0.591,0.736,0.733,0.687,0.198
average_recall,0.591,0.162,0.226,0.284,0.400,0.405,0.501,0.435,0.399
average_f1-score,0.522,0.198,0.246,0.258,0.477,0.522,0.595,0.531,0.265
average_NDCG,0.659,0.181,0.284,0.266,0.482,0.552,0.721,0.585,0.340
average_M1,0.605,0.163,0.256,0.209,0.442,0.558,0.767,0.589,0.163
average_M3,0.698,0.186,0.349,0.279,0.512,0.628,0.860,0.667,0.302
average_M5,0.744,0.209,0.349,0.279,0.512,0.628,0.884,0.675,0.442
average_position,1.406,1.444,1.333,1.333,1.136,1.148,1.184,1.156,2.700
average_length (x of gs),1.529,0.536,0.880,1.462,0.869,0.601,0.782,0.751,2.131


Now let us combine the individual results of the apps.

In [54]:
combinedResults = {}
for label, keys in combineLabels.items():
    collectedResults = {}
    for key in keys:
        result = results[key]
        app = key.split('_')[2]
        collectedResults[app] = result.loc[:,app]
    combinedResults[label] = pd.DataFrame(
        collectedResults,
        index=[
            f"average_{x}" for x in [
                "precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length (x of gs)"
            ]
        ]
        )

In [55]:
displayResults(combinedResults,printNumCases=False)

Results for experiment ignore_common_session_failed is


,Ada,Babylon,Buoy,K health,WebMD
average_precision,0.488,0.056,0.368,0.442,0.264
average_recall,0.633,0.034,0.354,0.465,0.537
average_f1-score,0.551,0.042,0.361,0.453,0.354
average_NDCG,0.718,0.039,0.419,0.538,0.507
average_M1,0.640,0.037,0.359,0.472,0.332
average_M3,0.833,0.042,0.525,0.606,0.545
average_M5,0.874,0.047,0.525,0.632,0.655
average_position,1.438,1.400,1.386,1.410,2.147
average_length (x of gs),1.486,0.567,1.008,1.194,2.217


Results for experiment ignore_common_no_disease_found is


,Ada,Babylon,Buoy,K health,WebMD
average_precision,0.488,0.373,0.422,0.388,0.266
average_recall,0.633,0.224,0.406,0.408,0.539
average_f1-score,0.551,0.280,0.414,0.398,0.356
average_NDCG,0.718,0.263,0.481,0.472,0.510
average_M1,0.640,0.250,0.413,0.414,0.333
average_M3,0.833,0.281,0.603,0.532,0.548
average_M5,0.874,0.312,0.603,0.555,0.658
average_position,1.438,1.400,1.386,1.410,2.147
average_length (x of gs),1.486,0.569,1.008,1.194,2.217


Results for experiment ignore_less-common_session_failed is


...

,Ada,Babylon,Buoy,K health,WebMD
average_precision,0.384,0.059,0.271,0.217,0.158
average_recall,0.512,0.029,0.268,0.273,0.330
average_f1-score,0.439,0.039,0.269,0.242,0.214
average_NDCG,0.516,0.016,0.278,0.222,0.266
average_M1,0.404,0.000,0.188,0.112,0.140
average_M3,0.556,0.000,0.333,0.213,0.247
average_M5,0.618,0.000,0.333,0.242,0.320
average_position,1.673,0.000,1.609,2.227,2.390
average_length (x of gs),1.647,0.380,1.075,1.367,2.355


Results for experiment ignore_all-cases_session_failed is


,Ada,Babylon,Buoy,K health,WebMD
average_precision,0.443,0.034,0.300,0.357,0.218
average_recall,0.581,0.020,0.291,0.399,0.446
average_f1-score,0.503,0.025,0.295,0.377,0.293
average_NDCG,0.630,0.023,0.331,0.413,0.401
average_M1,0.536,0.021,0.267,0.320,0.247
average_M3,0.712,0.023,0.410,0.447,0.413
average_M5,0.762,0.026,0.410,0.476,0.506
average_position,1.523,1.400,1.450,1.627,2.215
average_length (x of gs),1.557,0.512,1.037,1.271,2.279


Results for experiment ignore_all-cases_no_disease_found is


,Ada,Babylon,Buoy,K health,WebMD
average_precision,0.442,0.264,0.358,0.311,0.218
average_recall,0.579,0.157,0.348,0.348,0.446
average_f1-score,0.501,0.197,0.353,0.328,0.293
average_NDCG,0.628,0.177,0.395,0.360,0.401
average_M1,0.535,0.163,0.318,0.279,0.247
average_M3,0.710,0.184,0.489,0.389,0.413
average_M5,0.760,0.204,0.489,0.415,0.506
average_position,1.523,1.400,1.450,1.627,2.215
average_length (x of gs),1.557,0.518,1.036,1.271,2.279
